In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
from IPython import display

In [2]:
from ppo import PolicyNetwork, PPOAgent

In [3]:

# Environment setup
env = gym.make('CartPole-v1')  # Replace with your 2D maze environment


In [4]:

# PPO parameters
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.001
gamma = 0.99
epsilon_clip = 0.1

ppo_agent = PPOAgent(state_dim, action_dim, lr, gamma, epsilon_clip)


In [5]:

# Training loop
num_episodes = 1000

for episode in range(num_episodes):
    state = env.reset() # a tuple; the first element is the state
    state = state[0]
    done = False
    total_reward = 0
    states, actions, rewards, old_probs, values, dones = [], [], [], [], [], []

    while not done:
        # turn the state into a tensor with batch 1
        state = np.array(state)
        state = torch.FloatTensor(state).unsqueeze(0)
        # Collect data
        action_probs = ppo_agent.policy(state)
        action = torch.multinomial(action_probs, 1).item()
        value = ppo_agent.policy(state).detach().numpy()[0]
        next_state, reward, done, truncated, info = env.step(action)

        states.append(state)
        actions.append(action)
        rewards.append(reward)
        dones.append(done)

        old_probs.append(action_probs)
        values.append(value)

        state = next_state
        total_reward += reward

    # convert lists to tensors
    states = torch.cat(states)
    actions = torch.LongTensor(actions)
    old_probs = torch.cat(old_probs)
    values = torch.FloatTensor(values)
    dones = torch.FloatTensor(dones)

    # Compute returns and advantages
    returns = []
    advantages = ppo_agent.compute_advantage(rewards, values, dones)

    for t in range(len(rewards)):
        Gt = np.sum([r * (ppo_agent.gamma ** i) for i, r in enumerate(rewards[t:])])
        returns.append(Gt)

    # Normalize advantages
    advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-8)

    # Update policy
    ppo_agent.update_policy(states, actions, old_probs, advantages, returns)

    # Print episode info
    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")


/home/alex/anaconda3/envs/RL1/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/tmp/ipykernel_18116/1214111845.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  values = torch.FloatTensor(values)
/home/alex/anaconda3/envs/RL1/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([22, 2])) that is different to the input size (torch.Size([22])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([22, 2])
torch.Size([22, 2])
torch.Size([22, 2])
torch.Size([22, 2])
torch.Size([22])
torch.Size([22, 2])


RuntimeError: The size of tensor a (22) must match the size of tensor b (2) at non-singleton dimension 1